# Magister en Ciencia de Datos - UDD
## DBAnalytics (Ciencia de Datos aplicada)
**Sprint 2: Datos transformados II**

En base a las funciones RFM del Sprint 1, crear nuevos features para cada variable en el dataset user_logs.csv y transactions.csv (sólo si aplica).
Crear nuevas funciones para construir features tales como:
Tendencias de actividad del msno entre diferentes rangos de tiempo usando diferentes variables, usando variables del dataset user_logs.csv
Métricas de la actividad del msno usando diferentes variables del dataset transactions.csv 
En base a las funciones implementadas (y otras definidas por cada grupo), se deben implementar al menos 300 features. 

In [1]:
import pandas as pd
import numpy as np
import gc

* Lectura de archivos desde repositorio y contatenacion con nuevas versiones

In [2]:
!gsutil ls -a gs://kk_data_udd

gs://kk_data_udd/day_listen.csv#1564861559107461
gs://kk_data_udd/df_test.csv#1565240427109815
gs://kk_data_udd/df_train.csv#1565240406697715
gs://kk_data_udd/members_v3.csv#1563566790239785
gs://kk_data_udd/sample_submission_v2.csv#1563580288727022
gs://kk_data_udd/sample_submission_zero.csv#1563580145138161
gs://kk_data_udd/sub_age_xgb_pred.csv#1565055727433942
gs://kk_data_udd/sub_day_listen.csv#1564861658307683
gs://kk_data_udd/sub_reg_via_xgb_pred.csv#1565055581900599
gs://kk_data_udd/sub_user_satisfaction.cvs#1564861608790636
gs://kk_data_udd/test_sorted_v1.csv#1565242819914404
gs://kk_data_udd/train.csv#1563565831541482
gs://kk_data_udd/train_sorted_v1.csv#1565243444686022
gs://kk_data_udd/train_v2.csv#1563580263806878
gs://kk_data_udd/transactions.csv#1563580088583483
gs://kk_data_udd/transactions_v2.csv#1563580288931202
gs://kk_data_udd/user_label_201702.csv#1563681052454642
gs://kk_data_udd/user_label_201703.csv#1563681061521361
gs://kk_data_udd/user_latent_satisfaction.csv#1

* Leemos archivos de entrenamiento y test version 1.0

In [3]:
df_train = pd.read_csv('gs://kk_data_udd/train_sorted_v1.csv')

In [4]:
df_test = pd.read_csv('gs://kk_data_udd/test_sorted_v1.csv')

In [5]:
df_train[['city',
 'gender',
 'registered_via',
 'payment_method_id',
 'payment_plan_days',
 'plan_list_price',
 'discount',
 'membership_days']].head()

,city,gender,registered_via,payment_method_id,payment_plan_days,plan_list_price,discount,membership_days
0,18.0,2.0,9.0,38,30,149,0,0
1,10.0,1.0,9.0,39,30,149,0,0
2,11.0,2.0,9.0,39,30,149,0,0
3,13.0,2.0,9.0,40,30,149,0,0
4,3.0,1.0,9.0,38,90,477,0,0


In [6]:
def GetINT(obj):
    return str(obj).split('.')[0] 

In [7]:
def GetINT2(obj):
    return str(obj)

In [8]:
df_train['city'] = df_train['city'].apply(GetINT)
df_cities = pd.get_dummies(df_train[['city']], prefix=["city"])

df_train['gender'] = df_train['gender'].apply(GetINT)
df_gender = pd.get_dummies(df_train[['gender']], prefix=["gender"])

df_train['registered_via'] = df_train['registered_via'].apply(GetINT)
df_registered_via = pd.get_dummies(df_train[['registered_via']], prefix=["registered_via"])

In [9]:
df_train['payment_method_id'] = df_train['payment_method_id'].apply(GetINT2)
df_payment_method_id = pd.get_dummies(df_train[['payment_method_id']], prefix=["pmi"])

df_train['payment_plan_days'] = df_train['payment_plan_days'].apply(GetINT2)
df_payment_plan_days = pd.get_dummies(df_train[['payment_plan_days']], prefix=["ppd"])

df_train['plan_list_price'] = df_train['plan_list_price'].apply(GetINT2)
df_plan_list_price = pd.get_dummies(df_train[['plan_list_price']], prefix=["plp"])

df_train['discount'] = df_train['discount'].apply(GetINT2)
df_discount = pd.get_dummies(df_train[['discount']], prefix=["discount"])

df_train['membership_days'] = df_train['membership_days'].apply(GetINT2)
df_membership_days = pd.get_dummies(df_train[['membership_days']], prefix=["md"])

In [10]:
len(df_train.columns)

66

In [11]:
len(df_cities.columns)

22

In [12]:
df_train=pd.concat([df_train, df_cities],axis=1)

In [13]:
len(df_train.columns)

88

In [14]:
df_train=pd.concat([df_train, df_gender],axis=1)
df_train=pd.concat([df_train, df_registered_via],axis=1)
df_train=pd.concat([df_train, df_payment_method_id],axis=1)
df_train=pd.concat([df_train, df_payment_plan_days],axis=1)
df_train=pd.concat([df_train, df_plan_list_price],axis=1)
df_train=pd.concat([df_train, df_discount],axis=1)
df_train=pd.concat([df_train, df_membership_days],axis=1)

In [15]:
df_train.shape

(1963891, 217)

In [16]:
!gsutil ls -a gs://kk_data_udd

gs://kk_data_udd/day_listen.csv#1564861559107461
gs://kk_data_udd/df_test.csv#1565240427109815
gs://kk_data_udd/df_train.csv#1565240406697715
gs://kk_data_udd/members_v3.csv#1563566790239785
gs://kk_data_udd/sample_submission_v2.csv#1563580288727022
gs://kk_data_udd/sample_submission_zero.csv#1563580145138161
gs://kk_data_udd/sub_age_xgb_pred.csv#1565055727433942
gs://kk_data_udd/sub_day_listen.csv#1564861658307683
gs://kk_data_udd/sub_reg_via_xgb_pred.csv#1565055581900599
gs://kk_data_udd/sub_user_satisfaction.cvs#1564861608790636
gs://kk_data_udd/test_sorted_v1.csv#1565242819914404
gs://kk_data_udd/train.csv#1563565831541482
gs://kk_data_udd/train_sorted_v1.csv#1565243444686022
gs://kk_data_udd/train_v2.csv#1563580263806878
gs://kk_data_udd/transactions.csv#1563580088583483
gs://kk_data_udd/transactions_v2.csv#1563580288931202
gs://kk_data_udd/user_label_201702.csv#1563681052454642
gs://kk_data_udd/user_label_201703.csv#1563681061521361
gs://kk_data_udd/user_latent_satisfaction.csv#1

In [17]:
!gsutil ls -a gs://kk_data_udd/submission/

gs://kk_data_udd/submission/sub_day_listen_bins_pred.csv#1564861912140714
gs://kk_data_udd/submission/sub_orginal_day_listen_pred.csv#1564861906258198
gs://kk_data_udd/submission/sub_reg_via_xgb_pred.csv#1563929337202673


In [18]:
#user_latent_satisfaction
df_user_latent_satisfaction = pd.read_csv('gs://kk_data_udd/user_latent_satisfaction.csv')

In [19]:
df_user_latent_satisfaction.head()

,msno,user_latent_satisfaction
0,+2oK/qWmYvAnfNZsVV5pdsJ9n6d/LZn6CdwiJajGZas=,0.555549
1,+iMj+cgRiFJrmnOOLGwEZu6nzH0KbsMIdOZkpItAXTs=,0.798740
2,10QzA1bL7qK5Oeec6OlbyUEBOipzjY4OHN8gf2xokW4=,0.512925
3,2Dvm0YBqp+gDTOksgoR5ixNOk64C6ozpBzEkhbdkTrE=,0.958960
4,7B2xf5mD+GijSROhxSmKDuybhy+H5rgjfSy9sy6Xt4g=,0.762336


In [20]:
# day_listen
df_day_listen = pd.read_csv('gs://kk_data_udd/day_listen.csv')

In [21]:
df_day_listen.head()

,msno,is_churn,day_listen
0,+MFM2WIg07EADKRI0gm/Vq+3s2jDW838pVt477htI8k=,False,74.0
1,/fBID42G+PGSb8jQ4Lo0p6DI6gX78OIp5/thoFYn68M=,False,27.0
2,/qu3BCDcv1UqIHl2kal9kqrWlpYy4QMW5LolR5fR5/A=,False,126.0
3,/reu7jVe3U4hlk366ZmiR/YkVi6H/C2SSa8J4tSQtsQ=,False,66.0
4,1VIbdCaMTd1AKazZRzQDDrtO+emWOrncVaGRxEG8eiY=,False,130.0


In [22]:
del df_day_listen['is_churn']

In [23]:
df_train = pd.merge(df_train, df_day_listen, on='msno')

In [24]:
df_train = pd.merge(df_train,df_user_latent_satisfaction, on='msno')

In [25]:
df_train.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,...,discount_120,discount_149,discount_180,discount_20,discount_30,discount_50,md_-1,md_0,day_listen,user_latent_satisfaction
0,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10,38.0,1,9,2005.0,39,30,149,...,0,0,0,0,0,0,0,1,33.0,0.820694
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10,38.0,1,9,2005.0,39,30,149,...,0,0,0,0,0,0,0,1,33.0,0.820694
2,Nb1ZGEmagQeba5E+nQj8VlQoWl+8SFmLZu+Y8ytIamw=,1,18,22.0,2,9,2006.0,38,30,149,...,0,0,0,0,0,0,0,1,152.0,0.919394
3,Nb1ZGEmagQeba5E+nQj8VlQoWl+8SFmLZu+Y8ytIamw=,0,18,22.0,2,9,2006.0,38,30,149,...,0,0,0,0,0,0,0,1,152.0,0.919394
4,MkuWz0Nq6/Oq5fKqRddWL7oh2SLUSRe3/g+XmAWqW1Q=,1,11,31.0,2,9,2006.0,38,30,149,...,0,0,0,0,0,0,0,1,4.0,1.000000


In [26]:
df_train.shape

(1667562, 219)

In [27]:
df_train.to_csv("gs://kk_data_udd/train_sorted_v2.csv", index=False)